# Exploratory Data Analysis

## Setup

Download datasets to work directly from Google Colaboratory to preprocess the data and install libraries.

In [ ]:
!wget -P . -c https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv &> /dev/null
!wget -P . -c https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2022/2022-07-12/flights.csv &> /dev/null
!wget -P . -c https://raw.githubusercontent.com/ip2location/ip2location-iata-icao/master/iata-icao.csv &> /dev/null
!wget -P . -c https://raw.githubusercontent.com/com-480-data-visualization/project-2023-dqw4w9wgxcq/master/docs/res/data/cases_data.geojson &> /dev/null

In [ ]:
!pip install geojson

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Define imports, load Google Drive and move to project root path.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Enrich COVID data

In [ ]:
# Read the dataframes
covid_df = pd.read_csv('owid-covid-data.csv')

In [ ]:
# Change dateformat and show covid data format
covid_df['date']= pd.to_datetime(covid_df['date'])
covid_df["year"] = covid_df['date'].dt.year
covid_df["month"] = covid_df['date'].dt.month
covid_df["day"] = covid_df['date'].dt.day

covid_df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million,year,month,day
0,AFG,Asia,Afghanistan,2020-01-03,NaN,0.0,NaN,NaN,0.0,NaN,...,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN,2020,1,3
1,AFG,Asia,Afghanistan,2020-01-04,NaN,0.0,NaN,NaN,0.0,NaN,...,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN,2020,1,4
2,AFG,Asia,Afghanistan,2020-01-05,NaN,0.0,NaN,NaN,0.0,NaN,...,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN,2020,1,5
3,AFG,Asia,Afghanistan,2020-01-06,NaN,0.0,NaN,NaN,0.0,NaN,...,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN,2020,1,6
4,AFG,Asia,Afghanistan,2020-01-07,NaN,0.0,NaN,NaN,0.0,NaN,...,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN,2020,1,7


Set columns that you want to take into consideration

In [ ]:
columns = ["iso_code", "continent", "location", "year", "month", 'day', 'population', "total_cases", "new_cases", "new_cases_smoothed", "total_deaths",
           "new_deaths", "new_deaths_smoothed", "total_cases_per_million", "new_cases_per_million", "new_cases_smoothed_per_million",
           "total_deaths_per_million", "new_deaths_per_million", "new_deaths_smoothed_per_million"]

covid_df = covid_df[columns]

Keep only country codes in covid dataframe

In [ ]:
continents = ['OWID_AFR', 'OWID_ASI', 'OWID_EUR', 'OWID_OCE', 'OWID_SAM', 'OWID_NAM']
eu = ['OWID_EUN']
kosovo = ['OWID_KOS']
high_income = ['OWID_HIC']
low_income = ['OWID_LIC']
low_mid_income = ['OWID_LMC']
upper_mid_income = ['OWID_UMC']
north_cyprus = ['OWID_CYN']
uk = ['OWID_ENG', 'OWID_NIR', 'OWID_SCT', 'OWID_WLS']
world = ['OWID_WRL']
owid_codes = continents + eu + kosovo + high_income + low_income + low_mid_income + upper_mid_income + north_cyprus + uk + world

# Select unique country codes
iso_codes_covid = covid_df['iso_code'].unique()
iso_codes_covid = sorted(list(set(iso_codes_covid) - set(owid_codes)))

# Keep only country codes
covid_df = covid_df[covid_df.apply(lambda x: x['iso_code'] in iso_codes_covid, axis=1)]

Get neccessary COVID data

In [ ]:
# Get total deaths and total cases
covid_df[['iso_code', 'year', 'month', 'total_cases', 'total_deaths']].groupby(['iso_code']).last().sum()[-2:]

total_cases     766620422.0
total_deaths      6932670.0
dtype: float64

In [ ]:
# Get monthly total cases
cases_df = covid_df[['iso_code', 'year', 'month', 'total_cases', 'population']].groupby(['iso_code', 'year', 'month']).last()

# Proportion of deaths from country population
cases_df['value_total_cases'] = cases_df['total_cases'] / cases_df['population']
cases_df_min = cases_df.groupby(['iso_code']).agg({'value_total_cases': np.nanmin}).rename(columns={'value_total_cases': 'value_total_cases_min'})
cases_df_max = cases_df.groupby(['iso_code']).agg({'value_total_cases': np.nanmax}).rename(columns={'value_total_cases': 'value_total_cases_max'})
cases_df = cases_df.join(cases_df_min, on="iso_code")
cases_df = cases_df.join(cases_df_max, on="iso_code")

cases_df = cases_df.fillna(-1)
cases_df['value_total_cases_max_overall'] = cases_df['value_total_cases_max'].unique().max()

In [ ]:
monthly_data_df = covid_df[['iso_code', 'year', 'month', 'new_cases', 'new_deaths', 'population']].groupby(['iso_code', 'year', 'month']).sum()
monthly_data_df['new_cases_per_pop'] = monthly_data_df['new_cases'] / monthly_data_df['population']
monthly_data_df['new_deaths_per_pop'] = monthly_data_df['new_deaths'] / monthly_data_df['population']
monthly_data_df_min = monthly_data_df.groupby(['year', 'month']).agg({'new_cases_per_pop': np.nanmin}).rename(columns={'new_cases_per_pop': 'new_cases_per_pop_min'})
monthly_data_df_max = monthly_data_df.groupby(['year', 'month']).agg({'new_cases_per_pop': np.nanmax}).rename(columns={'new_cases_per_pop': 'new_cases_per_pop_max'})
monthly_data_df = monthly_data_df.join(monthly_data_df_min, on=["year", "month"])
monthly_data_df = monthly_data_df.join(monthly_data_df_max, on=["year", "month"])
monthly_data_df = monthly_data_df.fillna(-1)

In [ ]:
cases_df = pd.concat([cases_df, monthly_data_df], axis=1)
cases_df.head()

total_cases  population  value_total_cases  \
iso_code year month                                               
ABW      2020 1             -1.0    106459.0          -1.000000   
              2             -1.0    106459.0          -1.000000   
              3             50.0    106459.0           0.000470   
              4            100.0    106459.0           0.000939   
              5            101.0    106459.0           0.000949   

                     value_total_cases_min  value_total_cases_max  \
iso_code year month                                                 
ABW      2020 1                    0.00047               0.414995   
              2                    0.00047               0.414995   
              3                    0.00047               0.414995   
              4                    0.00047               0.414995   
              5                    0.00047               0.414995   

                     value_total_cases_max_overall  new_cases  new_deaths  \
iso_code year month                                                         
ABW      2020 1                           0.737555        0.0         0.0   
              2                           0.737555        0.0         0.0   
              3                           0.737555       50.0         0.0   
              4                           0.737555       50.0         2.0   
              5                           0.737555        1.0         1.0   

                     population  new_cases_per_pop  new_deaths_per_pop  \
iso_code year month                                                      
ABW      2020 1       3087311.0       0.000000e+00        0.000000e+00   
              2       3087311.0       0.000000e+00        0.000000e+00   
              3       3300229.0       1.515046e-05        0.000000e+00   
              4       3193770.0       1.565548e-05        6.262192e-07   
              5       3300229.0       3.030093e-07        3.030093e-07   

                     new_cases_per_pop_min  new_cases_per_pop_max  
iso_code year month                                                
ABW      2020 1                        0.0           9.449661e-07  
              2                        0.0           1.943223e-06  
              3                        0.0           2.395401e-04  
              4                        0.0           3.245276e-04  
              5                        0.0           4.905621e-04

Enrich GeoJSON

In [ ]:
from collections import defaultdict

mapping_monthly_cases = defaultdict(dict)
mapping_new_cases = defaultdict(dict)
mapping_new_cases_per_pop = defaultdict(dict)
mapping_new_deaths = defaultdict(dict)
mapping_new_deaths_per_pop = defaultdict(dict)

data = cases_df.to_dict()['value_total_cases']
for k, v in data.items():
    mapping_monthly_cases[k[0]].update({(f"{k[1]}_{k[2]}"): v})

data = cases_df.to_dict()['new_cases']
for k, v in data.items():
    mapping_new_cases[k[0]].update({(f"{k[1]}_{k[2]}"): v})

data = cases_df.to_dict()['new_cases_per_pop']
for k, v in data.items():
    mapping_new_cases_per_pop[k[0]].update({(f"{k[1]}_{k[2]}"): v})

data = cases_df.to_dict()['new_deaths']
for k, v in data.items():
    mapping_new_deaths[k[0]].update({(f"{k[1]}_{k[2]}"): v})

data = cases_df.to_dict()['new_deaths_per_pop']
for k, v in data.items():
    mapping_new_deaths_per_pop[k[0]].update({(f"{k[1]}_{k[2]}"): v})


In [ ]:
import geojson

with open('cases_data.geojson') as f:
    gj = geojson.load(f)
features = gj['features']

for f in features:
    iso_code = f['properties']['ISO_A3']
    if iso_code in ['PRK', 'TWN', 'TKM']:
        continue
    country_data_total_cases = mapping_monthly_cases.get(iso_code, None)
    country_data_new_cases = mapping_new_cases_per_pop.get(iso_code, None)

    if country_data_total_cases is not None:
        f['properties']['covidHeatmapTotalCases'] = country_data_total_cases
        f['properties']['covidHeatmapNewCases'] = country_data_new_cases
        f['properties']['covidHeatmapTotalCasesMin'] = cases_df.loc[iso_code]['value_total_cases_min'].iloc[0]
        f['properties']['covidHeatmapTotalCasesMax'] = cases_df.loc[iso_code]['value_total_cases_max'].iloc[0]

        for i in range(2018, 2024): 
            for j in range(1, 13):
                try:
                    d = cases_df.xs(i, level=1, drop_level=False).xs(j, level=2, drop_level=False)
                except:
                    continue
                f['properties'][f'covidHeatmapNewCasesMin_{i}_{j}'] = d.iloc[0]['new_cases_per_pop_min']
                f['properties'][f'covidHeatmapNewCasesMax_{i}_{j}'] = d.iloc[0]['new_cases_per_pop_max']

        f['properties']['covidHeatmapTotalCasesMaxOverall'] = cases_df.loc[iso_code]['value_total_cases_max_overall'].iloc[0]
        f['properties']['covidHeatmapNewCasesMaxOverall'] = cases_df['new_cases_per_pop'].max()

for f in features:
    iso_code = f['properties']['ISO_A3']
    if iso_code in ['PRK', 'TWN', 'TKM']:
        continue
    country_data = mapping_new_cases.get(iso_code, None)
    if country_data is not None:
        f['properties']['covidCasesNew'] = country_data

for f in features:
    iso_code = f['properties']['ISO_A3']
    if iso_code in ['PRK', 'TWN', 'TKM']:
        continue
    country_data = mapping_new_deaths.get(iso_code, None)
    if country_data is not None:
        f['properties']['covidDeathsNew'] = country_data

gj['features'] = features

In [ ]:
from geojson import dump

with open('cases_data.geojson', 'w') as f:
   dump(gj, f)

## Manipulate flights dataset

In [ ]:
flights_csv_output = 'all_agg_flights.csv'

In [ ]:
flights_df = pd.read_csv('flights.csv')
flights_df['APT_COUNTRY'] = flights_df['STATE_NAME']
flights_df['DATE'] = pd.to_datetime(flights_df['FLT_DATE'],
                                    format='%Y-%m-%dT%H:%M:%SZ')
flights_df['DAY'] = flights_df['DATE'].dt.day
flights_df['MONTH'] = flights_df['MONTH_NUM']
iata_icao_df = pd.read_csv('iata-icao.csv') \
                 .rename(columns=lambda _: 'APT_' + _.upper())

In [ ]:
flights_iata_icao_df = flights_df.merge(
    iata_icao_df[~iata_icao_df['APT_IATA'].isna() &
                 ~iata_icao_df['APT_ICAO'].isna() &
                 ~iata_icao_df['APT_COUNTRY_CODE'].isna()],
    on='APT_ICAO', how='inner'
)
flights_iata_icao_df = flights_iata_icao_df[['YEAR', 'MONTH', 'DAY', 'DATE', 
                                             'FLT_DEP_1', 'FLT_ARR_1',
                                             'FLT_TOT_1',
                                             'APT_ICAO', 'APT_IATA',
                                             'APT_COUNTRY', 'APT_COUNTRY_CODE',
                                             'APT_REGION_NAME',
                                             'APT_NAME', 'APT_AIRPORT',
                                             'APT_LATITUDE', 'APT_LONGITUDE']]
flights_iata_icao_df

,YEAR,MONTH,DAY,DATE,FLT_DEP_1,FLT_ARR_1,FLT_TOT_1,APT_ICAO,APT_IATA,APT_COUNTRY,APT_COUNTRY_CODE,APT_REGION_NAME,APT_NAME,APT_AIRPORT,APT_LATITUDE,APT_LONGITUDE
0,2016,1,1,2016-01-01,4,3,7,EBAW,ANR,Belgium,BE,Antwerpen,Antwerp,Antwerp International Airport,51.189400,4.46028
1,2016,1,2,2016-01-02,9,11,20,EBAW,ANR,Belgium,BE,Antwerpen,Antwerp,Antwerp International Airport,51.189400,4.46028
2,2016,1,3,2016-01-03,10,17,27,EBAW,ANR,Belgium,BE,Antwerpen,Antwerp,Antwerp International Airport,51.189400,4.46028
3,2016,1,4,2016-01-04,18,17,35,EBAW,ANR,Belgium,BE,Antwerpen,Antwerp,Antwerp International Airport,51.189400,4.46028
4,2016,1,5,2016-01-05,12,13,25,EBAW,ANR,Belgium,BE,Antwerpen,Antwerp,Antwerp International Airport,51.189400,4.46028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673720,2022,5,27,2022-05-27,3,3,6,LFOP,URO,France,FR,Normandie,Rouen,Rouen Airport,49.384201,1.17480
673721,2022,5,28,2022-05-28,4,6,10,LFOP,URO,France,FR,Normandie,Rouen,Rouen Airport,49.384201,1.17480
673722,2022,5,29,2022-05-29,7,6,13,LFOP,URO,France,FR,Normandie,Rouen,Rouen Airport,49.384201,1.17480
673723,2022,5,30,2022-05-30,5,5,10,LFOP,URO,France,FR,Normandie,Rouen,Rouen Airport,49.384201,1.17480


In [ ]:
# Airport codes: APT_ICAO, APT_IATA
# We can group by country, maybe region (if there are more than two airports)
# For display I have both the City - Neighbour and Airport Name, and position
agg_flights_df = flights_iata_icao_df.groupby(['YEAR', 'APT_IATA']) \
                     ['APT_AIRPORT', 'APT_COUNTRY',
                      'APT_LATITUDE', 'APT_LONGITUDE'] \
                    .agg(lambda _: _.iloc[0]) \
                    .join(flights_iata_icao_df.groupby(['YEAR', 'APT_IATA'])['FLT_TOT_1'] \
                                              .agg('sum').to_frame()) \
                    .loc[[2017, 2018, 2019, 2020, 2021]]

# agg_flights_df['FLT_TOT_1_NORMALIZED'] = 
agg_flights_df = (agg_flights_df.reset_index().merge(agg_flights_df.groupby(['YEAR'])['FLT_TOT_1'].max(), on="YEAR").set_index(['YEAR', 'APT_IATA']))
agg_flights_df["FLT_TOT_1_y"] = agg_flights_df["FLT_TOT_1_x"] / agg_flights_df["FLT_TOT_1_y"]
agg_flights_df = agg_flights_df.rename(columns={"FLT_TOT_1_x": "FLT_TOT_1_ORIG", "FLT_TOT_1_y": "FLT_TOT_1_NORMALIZED"})
agg_flights_df

<ipython-input-44-874fe3faceb2>:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  agg_flights_df = flights_iata_icao_df.groupby(['YEAR', 'APT_IATA']) \


APT_AIRPORT  \
YEAR APT_IATA                                                      
2017 ABC                                        Albacete Airport   
     ABZ                                        Aberdeen Airport   
     ACE                                       Lanzarote Airport   
     ADB                                  Adnan Menderes Airport   
     AGF                                 Agen La Garenne Airport   
...                                                          ...   
2021 XRY                        Jerez Airport (La Parra Airport)   
     ZAG                                   Franjo Tudman Airport   
     ZAZ                                        Zaragoza Airport   
     ZRH                                          Zurich Airport   
     ZTH       Zakynthos International Airport (Dionysios Sol...   

                  APT_COUNTRY  APT_LATITUDE  APT_LONGITUDE  FLT_TOT_1_ORIG  \
YEAR APT_IATA                                                                
2017 ABC                Spain     38.948502      -1.863520             836   
     ABZ       United Kingdom     57.201900      -2.197780           62587   
     ACE                Spain     28.945499     -13.605200           58004   
     ADB              Türkiye     38.292400      27.157000           82815   
     AGF               France     44.174702       0.590556            4765   
...                       ...           ...            ...             ...   
2021 XRY                Spain     36.744598      -6.060110           12642   
     ZAG              Croatia     45.742901      16.068800           28952   
     ZAZ                Spain     41.666199      -1.041550            8305   
     ZRH          Switzerland     47.464699       8.549170          125694   
     ZTH               Greece     37.750900      20.884300            9371   

               FLT_TOT_1_NORMALIZED  
YEAR APT_IATA                        
2017 ABC                   0.001645  
     ABZ                   0.123130  
     ACE                   0.114114  
     ADB                   0.162926  
     AGF                   0.009374  
...                             ...  
2021 XRY                   0.045170  
     ZAG                   0.103447  
     ZAZ                   0.029674  
     ZRH                   0.449109  
     ZTH                   0.033483  

[1536 rows x 6 columns]

In [ ]:
grouped1 = agg_flights_df[['FLT_TOT_1_ORIG', 'FLT_TOT_1_NORMALIZED']].unstack(level='YEAR')
grouped1.columns = grouped1.columns.map(lambda _: f'{_[0]}_{_[1]}').str.strip('|')
flights_iata_icao_df_no2022 = flights_iata_icao_df.drop(flights_iata_icao_df[flights_iata_icao_df['YEAR'] == 2022].index)
grouped2 = flights_iata_icao_df_no2022.groupby(['YEAR', 'MONTH', 'APT_IATA'])['FLT_TOT_1'].agg('sum').to_frame().unstack(level='YEAR').unstack(level='MONTH')
grouped2.columns = grouped2.columns.map(lambda _: f'{_[0]}_{_[1]}_{_[2]}').str.strip('|')
flights_iata_icao_df_final = flights_iata_icao_df_no2022.groupby(['APT_IATA']) \
                     ['APT_AIRPORT', 'APT_COUNTRY',
                      'APT_LATITUDE', 'APT_LONGITUDE'] \
                    .agg(lambda _: _.iloc[0]) \
                    .join(grouped1) \
                    .join(grouped2)
flights_iata_icao_df_final.to_csv(flights_csv_output)

<ipython-input-55-fe0279a8879a>:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  flights_iata_icao_df_final = flights_iata_icao_df_no2022.groupby(['APT_IATA']) \
